## 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn import svm,metrics
from sklearn.externals import joblib

## 데이터 불러오기

In [ ]:
Fold = 5

# k-fold 데이터
for i in range(Fold):
    
    j = i+1
    path1 = './SavedData/Training_Fold%d'%j
    path2 = './SavedData/Validation_Fold%d'%j
    c1 = 'Training_%d = np.array(pd.read_csv(path1, sep = ",", header = None))'%j
    c2 = 'Validation_%d = np.array(pd.read_csv(path2, sep = ",", header = None))'%j
    exec(c1)
    exec(c2)


# 전체 학습용 데이터
Training_TotalData = np.array(pd.read_csv('./SavedData/Training_TotalData', sep = ",", header = None))
Training_TotalLabel = np.array(pd.read_csv('./SavedData/Training_TotalLabel', sep = ",", header = None))

## Support Vector Machine 라벨링

In [ ]:
DataNo = 100

Training_Label_size = int((DataNo-DataNo/Fold))
Validation_Label_size = int((DataNo/Fold))

Training_Label_svm = np.zeros(shape=2*Training_Label_size)
Validation_Label_svm = np.zeros(shape=2*Validation_Label_size)

# Training Label
Training_Label_svm[:Training_Label_size] = 0
Training_Label_svm[Training_Label_size:] = 1

# Validation Label
Validation_Label_svm[:Validation_Label_size] = 0
Validation_Label_svm[Validation_Label_size:] = 1

# 전체 학습용 데이터 라벨
Training_TotalLabel_svm = np.zeros(shape=DataNo*2)

Training_TotalLabel_svm[:DataNo] = 0
Training_TotalLabel_svm[DataNo:] = 1

## SVM(Support Vector Machine) 기반 모델링 및 성능평가

In [ ]:
Add = 0
Divide = 0

for i in np.arange(Fold):
    c1 = 'Temp_train = Training_%d'%(i+1)
    exec(c1)
    c2 = 'Temp_val = Validation_%d'%(i+1)
    exec(c2)    

    svmModel = svm.SVC(kernel='linear')
    svmModel.fit(Temp_train, Training_Label_svm)
    Predicted = np.array(svmModel.predict(Temp_val))
    c3 = 'svmscore_%d = metrics.accuracy_score(Validation_Label_svm,Predicted)'%(i+1)
    exec(c3)
        
    Add += metrics.accuracy_score(Validation_Label_svm,Predicted)
    Divide += 1
    
Avg_accuracy = Add/Divide

print(' Fold 1: {:.5f} \n Fold 2: {:.5f} \n Fold 3: {:.5f} \n Fold 4: {:.5f} \n Fold 5: {:.5f}'
      .format(svmscore_1, svmscore_2, svmscore_3, svmscore_4, svmscore_5))
print('\n Average accuracy: {:.5f}'.format(Avg_accuracy))

## SVM model 모델 생성(전체 데이터) 및 저장

In [ ]:
SVM_model = svm.SVC(kernel='linear')
SVM_model.fit(Training_TotalData, Training_TotalLabel_svm)

joblib.dump(SVM_model, './AI Model/SVM_model.plk')

## SVM model 불러와서 진단

In [ ]:
SVM_model_from_joblib = joblib.load('./AI Model/SVM_model.plk')
Prediction = SVM_model_from_joblib.predict(Validation_2)
Diagnosis_Result = pd.DataFrame(Prediction)
Diagnosis_Result